In [137]:
import pandas as pd
import numpy as np
# Data intake
cities = pd.read_csv("datasets/cities.csv")
stations = pd.read_csv("datasets/stations.csv")
tracks = pd.read_csv("datasets/tracks.csv")
lines = pd.read_csv("datasets/lines.csv")
track_lines = pd.read_csv("datasets/track_lines.csv")
station_lines = pd.read_csv("datasets/station_lines.csv")
systems = pd.read_csv("datasets/systems.csv")
modes = pd.read_csv("datasets/modes.csv")

In [138]:
# Select and rename columns needed for merging in one go
cities_info = cities[['id', 'country', 'name']].rename(
    columns={'id': 'city_id', 'name': 'city'})
lines_info = lines[['id', 'name', 'color', 'system_id', 'transport_mode_id']].rename(
    columns={'id': 'line_id', 'name': 'line_name', 'color': 'line_color'})
systems_info = systems[['id', 'name']].rename(
    columns={'id': 'system_id', 'name': 'system_name'})
track_lines_info = track_lines[['section_id', 'line_id', 'fromyear', 'toyear']]
station_lines_info = station_lines[[
    'station_id', 'line_id', 'fromyear', 'toyear']]

In [139]:
# get mode for each line from modes and then merge
modes_info = modes[['id', 'name']].rename(
    columns={'id': 'transport_mode_id', 'name': 'transport_mode_name'})
modes_info['transport_mode_name'] = modes_info['transport_mode_name'].replace(
    'default', 'Unspecified')
modes_info['transport_mode_name'] = modes_info['transport_mode_name'].str.replace(
    '_', ' ')
modes_info['transport_mode_name'] = modes_info['transport_mode_name'].str.title()
modes_info['transport_mode_name'] = modes_info['transport_mode_name'].replace(
    'Brt', 'BRT')

lines_info = lines_info.merge(modes_info, on='transport_mode_id', how='left')

In [140]:
# --- Process STATIONS ---
stations = stations.rename(
    columns={'id': 'station_id', 'name': 'station_name'})

# Merge
stations = (stations
            .merge(cities_info, on='city_id', how='left')
            .merge(station_lines_info, on='station_id', how='left')
            .merge(lines_info, on='line_id', how='left')
            .merge(systems_info, on='system_id', how='left')
            )

# Extract coordinates
stations['longitude'] = stations['geometry'].str.extract(
    r'POINT\(([^ ]+) [^)]+\)')[0].astype(float)
stations['latitude'] = stations['geometry'].str.extract(
    r'POINT\([^ ]+ ([^)]+)\)')[0].astype(float)

station_cols = ['station_id', 'station_name', 'geometry', 'longitude', 'latitude',
                'opening', 'closure', 'city_id', 'city', 'country',
                'line_id', 'line_name', 'line_color', 'system_id', 'system_name',
                'transport_mode_id', 'transport_mode_name', 'fromyear', 'toyear']
stations = stations[station_cols]

# deduplicate rows by station_id
print(stations.shape)
stations = stations.drop_duplicates(subset=['station_id'])   
print(stations.shape)


(44897, 19)
(36846, 19)


In [141]:
# --- Process TRACKS ---
tracks = tracks.rename(columns={'id': 'section_id'})

# Merge
tracks = (tracks
          .merge(cities_info, on='city_id', how='left')
          .merge(track_lines_info, on='section_id', how='left')
          .merge(lines_info, on='line_id', how='left')
          .merge(systems_info, on='system_id', how='left')
          )

track_cols = ['section_id', 'geometry', 'opening', 'closure', 'length',
              'line_id', 'line_name', 'line_color', 'system_id', 'system_name',
              'city_id', 'city', 'country', 'transport_mode_id', 'transport_mode_name', 'fromyear', 'toyear']
tracks = tracks[track_cols]

# deduplicate rows by section_id, line_id, system_id
print(tracks.shape)
tracks = tracks.drop_duplicates(subset=['section_id', 'line_id'])
print(tracks.shape)

(35534, 17)
(35534, 17)


In [142]:
stations.transport_mode_name.value_counts()

transport_mode_name
Heavy Rail         18597
Commuter Rail       5509
Light Rail          5248
Unspecified         2256
Tram                1906
BRT                 1445
Inter City Rail      482
People Mover         440
Bus                  108
High Speed Rail       59
Ferry                 45
Name: count, dtype: int64

In [143]:
# --- Handle modal filter ---

# Filter out Bus and Ferry
desel_modes = ['Bus', 'Ferry']
stations = stations[~stations['transport_mode_name'].isin(desel_modes)]
tracks = tracks[~tracks['transport_mode_name'].isin(desel_modes)]

In [144]:

# --- Handle name missing data ---

# Fill in NA
stations['station_name'] = stations['station_name'].fillna('')
stations['line_name'] = stations['line_name'].fillna('')
stations['line_color'] = stations['line_color'].fillna('#000000')
tracks['line_name'] = tracks['line_name'].fillna('')
tracks['line_color'] = tracks['line_color'].fillna('#000000')
stations['line_id'] = stations['line_id'].fillna(0)
tracks['line_id'] = tracks['line_id'].fillna(0)

# --- Handle opening and closure date missing data ---
stations.loc[stations['closure'] == 999999, 'closure'] = pd.NA
tracks.loc[tracks['closure'] == 999999, 'closure'] = pd.NA
# Handle Opening Dates, impute missing data based on minimum known opening year for each city
stations['opening'] = pd.to_numeric(stations['opening'], errors='coerce')
tracks['opening'] = pd.to_numeric(tracks['opening'], errors='coerce')
stations.loc[stations['opening'] == 0, 'opening'] = pd.NA
tracks.loc[tracks['opening'] == 0, 'opening'] = pd.NA
stations['opening'] = stations['opening'].astype('Int64')
tracks['opening'] = tracks['opening'].astype('Int64')
# Flag rows still missing opening date
stations['was_missing_opening'] = stations['opening'].isna()
tracks['was_missing_opening'] = tracks['opening'].isna()

In [145]:
# ... (wonkiness calculation) ...
# Calculate proportion of missing/imputed opening dates per city
pivot_st = pd.pivot_table(stations, values='station_id', index='city_id',
                          columns='was_missing_opening', aggfunc='count', fill_value=0)
pivot_tr = pd.pivot_table(tracks, values='section_id', index='city_id',
                          columns='was_missing_opening', aggfunc='count', fill_value=0)

pivot_st = pivot_st.rename(columns={True: 'missing', False: 'valid'})
pivot_tr = pivot_tr.rename(columns={True: 'missing', False: 'valid'})
if 'missing' not in pivot_st.columns:
    pivot_st['missing'] = 0
if 'valid' not in pivot_st.columns:
    pivot_st['valid'] = 0
if 'missing' not in pivot_tr.columns:
    pivot_tr['missing'] = 0
if 'valid' not in pivot_tr.columns:
    pivot_tr['valid'] = 0

# Calculate the "wonkiness" score 9missing dates) then merge for filtering
pivot_st['wonkiness_st'] = pivot_st['missing'] / \
    (pivot_st['valid'] + pivot_st['missing'])
pivot_tr['wonkiness_tr'] = pivot_tr['missing'] / \
    (pivot_tr['valid'] + pivot_tr['missing'])
wonk_table = pd.merge(pivot_st[['valid', 'missing', 'wonkiness_st']],
                      pivot_tr[['valid', 'missing', 'wonkiness_tr']],
                      on='city_id', suffixes=('_st', '_tr'))

wonk_table = wonk_table.merge(
    cities_info[['city_id', 'city', 'country']], on='city_id', how='left')
wonk_table['total_stations'] = wonk_table['valid_st'] + \
    wonk_table['missing_st']
wonk_table['total_tracks'] = wonk_table['valid_tr'] + wonk_table['missing_tr']

# wonk based on tracks only
wonk_table['avg_wonkiness'] = wonk_table['wonkiness_tr']

In [146]:
# --- Extract LineString coordinates from WKT strings ---
import re


def parse_linestring_coords(wkt_string, swap_order=False, precision=6):
    if not isinstance(wkt_string, str):
        return None

    # Regular expression to find all coordinate pairs (lon lat)
    coord_pattern = re.compile(r"(-?\d+\.?\d*)\s+(-?\d+\.?\d*)")
    matches = coord_pattern.findall(wkt_string)
    if not matches:
        return None

    coord_list = []
    for lon_str, lat_str in matches:
        try:
            lon = round(float(lon_str), precision)
            lat = round(float(lat_str), precision)
            if swap_order:
                coord_list.append([lat, lon])  # Append as [lat, lon]
            else:
                coord_list.append([lon, lat])  # Append as [lon, lat]
        except ValueError:
            print(
                f"WARN: Could not parse coordinate pair: ('{lon_str}', '{lat_str}') in '{wkt_string}'")
            continue

    return coord_list if coord_list else None


tracks['linestring_lonlat'] = tracks['geometry'].apply(
    lambda x: parse_linestring_coords(x, swap_order=False)
)  # to be used for export
tracks['linestring_latlon'] = tracks['geometry'].apply(
    lambda x: parse_linestring_coords(x, swap_order=True)
)  # to be used for map plotting dl

In [148]:
import pandas as pd
import numpy as np

def canonicalize_years(df):
    year_cols = ['opening', 'closure', 'fromyear', 'toyear']

    # Convert potentially present year columns to numeric, errors become NaN
    for col in year_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Prioritize 'fromyear' for 'opening'
    if 'fromyear' in df.columns:
        if 'opening' in df.columns:
            df['opening'] = df['fromyear'].combine_first(df['opening'])
        else:
            df['opening'] = df['fromyear']

    # Prioritize 'toyear' for 'closure'
    if 'toyear' in df.columns:
        if 'closure' in df.columns:
            df['closure'] = df['toyear'].combine_first(df['closure'])
        else:
            df['closure'] = df['toyear']

    if 'opening' in df.columns:
        df.loc[df['opening'] <= 0, 'opening'] = np.nan
    if 'closure' in df.columns:
        df.loc[df['closure'] <= 0, 'closure'] = np.nan

    return df


# Apply the function
stations = canonicalize_years(stations.copy()) 
tracks = canonicalize_years(tracks.copy())

In [149]:
# export
stations.to_csv(
    "stations_cleaned.csv",
    index=False,
    encoding='utf-8',
)
tracks.to_csv(
    "tracks_cleaned.csv",
    index=False,
    encoding='utf-8',
)
wonk_table.to_csv(
    "cities_cleaned.csv",
    index=True,
    encoding='utf-8',
)

In [150]:
quality_threshold = 0.1  # 10%
quantity_threshold = 100

filtered_cities = wonk_table[
    (wonk_table['avg_wonkiness'] <= quality_threshold) &
    (wonk_table['total_stations'] >= quantity_threshold)
]
print(quality_threshold)
print(quantity_threshold)
print(filtered_cities.shape)
filtered_cities[['city', 'avg_wonkiness', 'total_stations']
                ].sort_values('total_stations', ascending=False)

0.1
100
(48, 12)


,city,avg_wonkiness,total_stations
63,Tokyo,0.000897,3715
93,New York,0.096032,1339
49,Osaka,0.000000,1333
33,London,0.007910,907
161,Seoul,0.000000,876
50,Paris,0.008214,827
0,Buenos Aires,0.019417,793
44,Nagoya,0.004246,787
8,Beijing,0.000000,572
35,Madrid,0.041667,560
